In [1]:
import copy

import pandas as pd
import numpy as np
import tensorflow

from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# Read in, Normalize and Split Data

In [13]:
target_variable = 'Strength'

In [14]:
df_cement = pd.read_csv('concrete_data.csv')

print(sorted(list(df_cement)))

['Age', 'Blast Furnace Slag', 'Cement', 'Coarse Aggregate', 'Fine Aggregate', 'Fly Ash', 'Strength', 'Superplasticizer', 'Water']


In [15]:

def normalize(data_to_normalize):
    data_to_normalize_scaled = copy.copy(data_to_normalize)
    for feature in data_to_normalize.columns:
        mean_value_of_feature = np.mean(data_to_normalize[feature].values)
        std_dev_value_of_feature = np.std(data_to_normalize[feature].values)

        data_to_normalize_scaled[feature] = (data_to_normalize[feature] - mean_value_of_feature) / std_dev_value_of_feature

    return data_to_normalize_scaled

In [17]:
# Create a features and target dataframe, where df_features does not contain target variable and df_target contains only target variable
df_features = df_cement.drop(target_variable,  axis=1)

df_target = df_cement[target_variable]

df_features_normalized = normalize(data_to_normalize=df_features)
df_features_normalized

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,0.863154,-1.217670,-0.279733
1,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,1.056164,-1.217670,-0.279733
2,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,3.553066
3,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,5.057677
4,-0.790459,0.678408,-0.847144,0.488793,-1.039143,0.070527,0.647884,4.978487
...,...,...,...,...,...,...,...,...
1025,-0.045645,0.488235,0.564545,-0.092171,0.451410,-1.323005,-0.065893,-0.279733
1026,0.392819,-0.856888,0.960068,0.676200,0.702626,-1.994680,0.496893,-0.279733
1027,-1.270088,0.759579,0.850635,0.521589,-0.017528,-1.036064,0.080107,-0.279733
1028,-1.168610,1.308065,-0.847144,-0.279579,0.853356,0.214641,0.191166,-0.279733


In [18]:
#Obtain split data in dataframe format, by sending df_features and df_target to helper function
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features_normalized, df_target, test_size=0.3)

# Build Keras Model

In [19]:
input_shape = (df_features_train.shape[1],)

## Create Model Architecture

In [20]:
model = Sequential()
#Single hidden layer, 
model.add(Dense(10, input_shape=input_shape, activation="relu"))
model.add(Dense(1))

# Compile and Fit Model

In [21]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [22]:
model.fit(df_features_train, df_target_train, epochs=100)

Epoch 1/100
23/23 [==============================] - 0s 517us/step - loss: 1533.4612 - mse: 1533.4612
Epoch 2/100
23/23 [==============================] - 0s 471us/step - loss: 1513.4938 - mse: 1513.4938
Epoch 3/100
23/23 [==============================] - 0s 488us/step - loss: 1493.8890 - mse: 1493.8890
Epoch 4/100
23/23 [==============================] - 0s 512us/step - loss: 1474.1556 - mse: 1474.1556
Epoch 5/100
23/23 [==============================] - 0s 451us/step - loss: 1454.2842 - mse: 1454.2842
Epoch 6/100
23/23 [==============================] - 0s 476us/step - loss: 1434.8541 - mse: 1434.8541
Epoch 7/100
23/23 [==============================] - 0s 456us/step - loss: 1414.7136 - mse: 1414.7136
Epoch 8/100
23/23 [==============================] - 0s 526us/step - loss: 1394.6445 - mse: 1394.6445
Epoch 9/100
23/23 [==============================] - 0s 467us/step - loss: 1374.2546 - mse: 1374.2546
Epoch 10/100
23/23 [==============================] - 0s 457us/step - loss: 1353.8

In [23]:
results = model.evaluate(x=df_features_test, y=df_target_test, return_dict=True)
print(results['mse'])

10/10 [==============================] - 0s 529us/step - loss: 165.2918 - mse: 165.2918
165.29180908203125


# Retrain Model Split and Train 50 Times

In [24]:
ls_mse_values = []

for iteration in range(0, 50):
    #Obtain split data in dataframe format, by sending df_features and df_target to helper function
    df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, test_size=0.3)

    # Create a features and target dataframe, where df_features does not contain target variable and df_target contains only target variable
    df_features = df_cement.drop(target_variable, axis=1)

    df_target = df_cement[target_variable]

    df_features_normalized = normalize(data_to_normalize=df_features)

    #Obtain split data in dataframe format, by sending df_features and df_target to helper function
    df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, test_size=0.3)

    input_shape = (df_features_train.shape[1],)

    #Create Model
    model = Sequential()
    #Single hidden layer, 
    model.add(Dense(10, input_shape=input_shape, activation="relu"))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    model.fit(df_features_train, df_target_train, epochs=100, verbose=0)

    results = model.evaluate(x=df_features_test, y=df_target_test, return_dict=True)

    ls_mse_values.append(results['mse'])

10/10 [==============================] - 0s 616us/step - loss: 52.3623 - mse: 52.3623


## Get Mean and Standard Deviations of 50 Runs

In [27]:
avg_mse = np.mean(np.array(ls_mse_values))
std_dev_mse = np.std(np.array(ls_mse_values))

print("For the 50 runs, the mean MSE is 344.12 with a standard deviation of 361.04")
print("For the 50 runs, the mean MSE is 293.03 with a standard deviation of 374.86")
print("For the 50 runs on normalzied data with 100 epochs, the mean MSE is {:.2f} with a standard deviation of {:.2f}".format(avg_mse, std_dev_mse))

For the 50 runs, the mean MSE is 344.12 with a standard deviation of 361.04
For the 50 runs, the mean MSE is 293.03 with a standard deviation of 374.86
For the 50 runs on normalzied data with 100 epochs, the mean MSE is 161.97 with a standard deviation of 89.87
